In [1]:
import os 


In [2]:
%pwd

'c:\\MyDLProject\\Kidney_DL_Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\MyDLProject\\Kidney_DL_Project'

In [5]:
### entity 

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
### config manager 

In [8]:
import mlflow
import os

# Set MLflow tracking URI
mlflow.set_tracking_uri("https://dagshub.com/Pradipchandanshive/Kidney_DL_Project.mlflow")

# Set your DagsHub credentials (MLflow uses HTTP Basic Auth)
os.environ["MLFLOW_TRACKING_USERNAME"] = "PradipChandanshive"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "fd872e30a5f152a078820e8a236b9d00765a27fa"

c:\Users\Dell\anaconda3\envs\kidney\Lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


In [9]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
### config manager 

In [11]:
from src.cnnClassifier.constants import *

from src.cnnClassifier.utils.common import read_yaml,create_directories,save_json


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Pradipchandanshive/Kidney_DL_Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        if len(self.valid_generator) > 0:
            self.score = self.model.evaluate(self.valid_generator)
        else:
            print("Validation generator is empty.")

        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print("MLflow URI:", mlflow.get_tracking_uri())
        print("MLflow Experiment:", mlflow.get_experiment_by_name("your_experiment_name"))
        print("Active run:", mlflow.active_run())
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 139 images belonging to 2 classes.


c:\Users\Dell\anaconda3\envs\kidney\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.3158 - loss: 2.4503
9/9 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - accuracy: 0.3158 - loss: 2.4503
MLflow URI: https://dagshub.com/Pradipchandanshive/Kidney_DL_Project.mlflow
MLflow Experiment: None
Active run: None


2025/05/12 16:58:42 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2025/05/12 16:59:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
